Cleaning Demographic Data

In [1]:
import itertools
import numpy as np
import import_ipynb
import pandas as pd
import datetime as dt
from patient_visits import demog2_461_014
from central_functions import read_files, handle_nan
from screen_fails import SCREEN_FAILS1, PATIENT_VISITS
from data_files import demog, myk_461_012, myk_461_018

importing Jupyter notebook from patient_visits.ipynb
importing Jupyter notebook from data_files.ipynb
importing Jupyter notebook from central_functions.ipynb
C:/Users/dgao/Box/Intern Data Models/461-012/dta_demographics.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_sbj_participation.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_subject_termination.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_time_table.txt
C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf
C:/Users/dgao/Box/Intern Data Models/461-012/myk461-012_visit.csv
C:/Users/dgao/Box/Intern Data Models/461-012/udp_pi_eligibility_statement.txt
importing Jupyter notebook from screen_fails.ipynb
0 C:/Users/dgao/Box/Intern Data Models/224-001/224_001_elig_enroll_log.csv
1 C:/Users/dgao/Box/Intern Data Models/224-001/elig_enrollment_log.pdf
2 C:/Users/dgao/Box/Intern Data Models/461-005/elig.sas7bdat
3 C:/Users/dgao/Box/Intern Data Models/

C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


SF before check:  (709, 12)
SF after check/dropping 'randomized' patients:  (677, 12)
Number of patient IDs to add to visits:  290
Num of rows in visits BEFORE rows added from SF:  9366
This is the new # of rows in visits (should be 305 more) 9656


In [2]:
def clean_vars_demog(df, bday, sex, s_asian, amer, ese_asian,**kwargs):
    clean_vars = df[['subjectId', 'Subject', bday, sex, 'ETHNIC','RACEAFR','RACEEUR', s_asian,'RACEPAC', amer,'RACEUNK', ese_asian]]
    for k in kwargs.items():
        clean_vars = pd.concat([clean_vars,pd.DataFrame(df[k[1]])],axis=1)
    return clean_vars                   

# DEMOGRAPHIC Standardizing Variable Names Across DFs
def standardize_vars_for_patients(df):
    col_names = list(df.columns)
    col_renames = ['subject_id','patient_id','dob','sex','race_hisp_latin','race_afr','race_eur','race_sasian','race_pi','race_amer','race_unkn', 'race_seasian']
    renames = list(zip(col_names,col_renames))
    print(list(renames))
    df = df.rename(columns={col_names[0]:col_renames[0],col_names[1]:col_renames[1],col_names[2]:col_renames[2],col_names[3]:col_renames[3],col_names[4]:col_renames[4],col_names[5]:col_renames[5],col_names[6]:col_renames[6],col_names[7]:col_renames[7],col_names[8]:col_renames[8],col_names[9]:col_renames[9],col_names[10]:col_renames[10],col_names[11]:col_renames[11]})
    return df

def replace_vals_for_patients(df):
    map = {"Not Hispanic or Latino" : 0, "Hispanic or Latino": 1, 'Not Reported': 2}
    df['race_hisp_latin'] = df.race_hisp_latin.map(map)
    return df

In [3]:
def run_all_for_patients(df):
    df = df.drop(['subjectId'],axis=1,errors='ignore')
    df.columns = map(str.lower, df.columns)
    df = df.rename(columns={'id_participation':'patient_id','subjectid':'patient_id','subject':'patient_id','usubjid':'patient_id','brthdat':'dob','brthdat_int':'dob','brthdatn':'dob','brthyr':'dob'})
    if 'study_name' in df.columns:
        if '244744.CAL' in df.study_name.values:
            df = df.drop('patient_id',axis=1)
            df = df.rename(columns={'id_subject':'patient_id'})
    df = df.dropna(axis=1,how='all')
    df = df.drop(['ethniccd','racecd','sexcd','sex_cv','sex_std','subjid','visdatn','visdatc','subj','dm_repeat','infcon_dsstdatc','infcon_dsstdatn','tivers','lastupdon','lastupdby','subjectkeylabel','studyevent','eventrepeatkey','formname','frep','formstate','rfstdtc','rfendtc','rfxstdtc','rfxendtc','rficdtc','rfpendtc','invnam','armcd','arm','actarmcd','actarm','dmdtc','dmdy','runin_no','id_demographics','valid','period_no','continuous_value','setup_version','trialday_no','period_no','id_rsl_procedure','period_kind','control_no','period_name','trialday_name','continuous_name','procedure_name','procedure_state_name','actual_dt','study_name','studyid','brthdat_c','brthdatc','brthdat_raw','brthdat_yyyy','brthdat_m','brthdat_mm','brthdat_dd','brthdat_d','projectid','project','environmentname','studysiteid','siteid','site','sitenumber','sitegroup','instanceid','instancename','instancerepeatnumber','folderid','folder','foldername','folderseq','targetdays','datapageid','datapagename','pagerepeatnumber','recordid','recordposition','recorddate','mincreated','maxupdated','savets','coder_hierarchy','studyenvsitenumber','siteseq','sitename','sitecode','subjectseq','eventseq','eventid','eventname','eventdate','activityid','activityname','formseq','subjectformseq','originsubjectformseq','sourcesubjectformseq','designversion','age_raw','ageu'],axis=1,errors='ignore')
    drop = []
    for col in df.columns:
        if '_raw' in col or '_cv' in col or '_std' in col:
            drop.append(col)
    df = df.drop(drop, axis=1, errors='ignore')
    if 'study_no_sponsor' in df.columns:
        for i,row in df.iterrows():
            df.loc[i,'patient_id'] = df.loc[i,'study_no_sponsor'][4:]+'-'+str(df.loc[i,'patient_id'])
        df = df.drop(['study_no_sponsor'],axis=1)
    return df

In [4]:
def final_subset(df):
    subset = ['patient_id','ethnic']
    if 'random_no' in df.columns:
        subset.append('random_no')
    if 'sex' in df.columns:
        subset.append('sex')
    for col in df.columns:
        if 'race' in col:
            subset.append(col)
    if 'dob' not in df.columns:
        df = df.rename(columns={'brthdat_y':'dob','frozen_age':'dob'},errors='ignore')
    if 'dob' in df.columns:
        subset.append('dob')
    if 'age' in df.columns:
        subset.append('age')
    return df[subset]

These demographic data come from the Rave MediData EDC system

These data represent patient demographic information, and there is one unique row per patient

In [5]:
#OG d Files
d_224_001 = read_files(demog[0],skiprows=1)
d_461_004 = read_files(demog[1])
d1_461_005 = read_files(demog[2])
d2_461_005 = read_files(demog[3])[['Subject','RACE2AS','RACE2AIA','RACE2BAA','RACE2NHO', 'RACE2WH', 'RACE2UNK']]
d1_461_006 = read_files(demog[4])
d2_461_006 = read_files(demog[5])[['Subject','RACEASIAN','RACEAMIND','RACEBLACK','RACEHAWPAC','RACEWHITE','RACEUNK']]
d_461_007 = read_files(demog[6])
# d_461_007.loc[219,'Subject'] = '461-005-902-501'
# d_461_007.loc[224,'Subject'] = '461-005-297-502'
# d_461_007.loc[225,'Subject'] = '461-005-297-501'
d_461_008 = read_files(demog[7])
d_461_012 = read_files(demog[8])
sex_461_012 = read_files(myk_461_012[1])[['random_no','sbj_sex']]
d_461_013 = read_files(demog[9])
d_461_014 = read_files(demog[10])
d_461_015 = read_files(demog[11])
d_461_018 = read_files(demog[12])
sex_461_018 = read_files(myk_461_018[1])[['ID_PARTICIPATION','SBJ_SEX']]
d_491_003_mad = read_files(demog[13])
d_491_003_sad = read_files(demog[14])

In [6]:
d1_461_005[d1_461_005.Subject=='461-005-205-505']
d2_461_005[d2_461_005.Subject=='461-005-205-505']
# d_461_005[d_461_005.Subject=='461-005-205-505']
# demog_461_005[demog_461_005.patient_id=='461-005-205-505']
t = read_files(demog[3])
t[t.Subject=='461-005-205-505']
myk_461_012
demog[8]

'C:/Users/dgao/Box/Intern Data Models/461-012/dta_demographics.txt'

In [56]:
d1_461_005
demog[2]

'C:/Users/dgao/Box/Intern Data Models/461-005/dm1.sas7bdat'

In [8]:
for i in range(len(demog)):
    print(i,demog[i])


0 C:/Users/dgao/Box/Intern Data Models/224-001/dm.xlsx
1 C:/Users/dgao/Box/Intern Data Models/461-004/dm1.sas7bdat
2 C:/Users/dgao/Box/Intern Data Models/461-005/dm1.sas7bdat
3 C:/Users/dgao/Box/Intern Data Models/461-005/dm2.sas7bdat
4 C:/Users/dgao/Box/Intern Data Models/461-006/dm1.sas7bdat
5 C:/Users/dgao/Box/Intern Data Models/461-006/dm2.sas7bdat
6 C:/Users/dgao/Box/Intern Data Models/461-007/dm1.sas7bdat
7 C:/Users/dgao/Box/Intern Data Models/461-008/dm1.sas7bdat
8 C:/Users/dgao/Box/Intern Data Models/461-012/dta_demographics.txt
9 C:/Users/dgao/Box/Intern Data Models/461-013/dm.sas7bdat
10 C:/Users/dgao/Box/Intern Data Models/461-014/dta_demographics.txt
11 C:/Users/dgao/Box/Intern Data Models/461-015/dm.sas7bdat
12 C:/Users/dgao/Box/Intern Data Models/461-018/dta_demographics.xlsx
13 C:/Users/dgao/Box/Intern Data Models/491-003/MAD Cohort/xdm_mad.sas7bdat
14 C:/Users/dgao/Box/Intern Data Models/491-003/SAD Cohort/xdm.sas7bdat


In [9]:
d_461_005 = d1_461_005.merge(d2_461_005,how='left',on='Subject')
d_461_006 = d1_461_006.merge(d2_461_006,how='left',on='Subject')

In [10]:
demog_224_001 = run_all_for_patients(d_224_001)
demog_461_004= run_all_for_patients(d_461_004)
demog_461_005 = run_all_for_patients(d_461_005)
demog_461_006 = run_all_for_patients(d_461_006)
demog_461_007 = run_all_for_patients(d_461_007)
demog_461_008 = run_all_for_patients(d_461_008)
demog_461_012 = run_all_for_patients(d_461_012)
demog_461_013 = run_all_for_patients(d_461_013)
demog_461_014 = run_all_for_patients(d_461_014)
demog_461_015 = run_all_for_patients(d_461_015)
demog_461_018 = run_all_for_patients(d_461_018)
demog_491_003_mad = run_all_for_patients(d_491_003_mad)
demog_491_003_sad = run_all_for_patients(d_491_003_sad)

In [11]:
measures2_224_001 = demog_224_001[['patient_id','dmcbp', 'dmcbpcd', 'f_dmbcm', 'f_dmbcmcd','m_dmbcm', 'm_dmbcmcd','dmess', 'dmesscd','dmrdyn', 'dmrdyncd']]
measures2_461_012 = demog_461_012[['patient_id','smoker_yn', 'smoke', 'smoke_duration','smoke_pieces', 'smoke_start_year', 'smoke_end_year', 'smoke_end_month','ex_smoker_yn', 'smoke_pack_years', 'drug_yn', 'alcohol_yn','alcohol_g_day', 'alcohol_g_week', 'alcohol_units_day','alcohol_units_week', 'beverages_yn', 'alcohol_remark','beverages_remark', 'blood_donate_yn', 'plasma_don_yn', 'vegetarian']]
measures2_461_014 = demog_461_014[['patient_id','smoker_yn', 'smoke', 'smoke_duration','smoke_pieces', 'smoke_start_year', 'smoke_start_month','smoke_end_year', 'smoke_end_month', 'ex_smoker_yn', 'smoke_current','smoke_pack_years', 'drug_yn', 'alcohol_yn', 'alcohol_g_day','alcohol_g_week', 'alcohol_units_day', 'alcohol_units_week','beverages_yn', 'alcohol_remark', 'beverages_remark', 'blood_donate_yn','plasma_don_yn', 'vegetarian']]
measures2_461_018 = demog_461_018[['patient_id','smoker_yn', 'smoke', 'smoke_duration','smoke_pieces', 'smoke_start_year', 'smoke_start_month','smoke_end_year', 'smoke_end_month', 'ex_smoker_yn', 'smoke_pack_years','drug_yn', 'alcohol_yn', 'alcohol_g_day', 'alcohol_g_week','alcohol_units_day', 'alcohol_units_week', 'beverages_yn','alcohol_remark', 'beverages_remark', 'blood_donate_yn','plasma_don_yn', 'vegetarian']]

In [12]:
patients_224_001 = final_subset(demog_224_001)
patients_461_004= final_subset(demog_461_004)
patients_461_005 = final_subset(demog_461_005)
patients_461_006 = final_subset(demog_461_006)
patients_461_007 = final_subset(demog_461_007)
patients_461_008 = final_subset(demog_461_008)
patients_461_012 = final_subset(demog_461_012)
patients_461_013 = final_subset(demog_461_013)
patients_461_014 = final_subset(demog_461_014)
patients_461_015 = final_subset(demog_461_015)
patients_461_018 = final_subset(demog_461_018)
patients_491_003_mad = final_subset(demog_491_003_mad)
patients_491_003_sad = final_subset(demog_491_003_sad)

In [13]:
dfs = [('224_001',patients_224_001),('461_004',patients_461_004),('461-005',patients_461_005),('461-006',patients_461_006),('461-007',patients_461_007),('461-008',patients_461_008),('461-012',patients_461_012),('461-013',patients_461_013),('461-014',patients_461_014),('461-015',patients_461_015),('461-018',patients_461_018),('491-003-MAD',patients_491_003_mad),('491-003-SAD',patients_491_003_sad)]

In [14]:
for i,row in patients_224_001.iterrows():
    if patients_224_001.loc[i,'race'] == 'OTHER':
        patients_224_001.loc[i,'race'] = patients_224_001.loc[i,'raceoth']
patients_224_001 = patients_224_001.drop(['raceoth'],axis=1)
patients_224_001['dob'] = patients_224_001.dob.apply(lambda x: str(x)+'-01-01')
patients_224_001['patient_id'] = patients_224_001.patient_id.apply(lambda x: '224-001-'+x)

C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
patients_224_001['dob'] = patients_224_001.dob.apply(lambda x: pd.to_datetime(x[:4]+x[5:7]+x[8:],format='%Y%m%d'))

In [16]:
patients_461_004['patient_id'] = patients_461_004.patient_id.apply(lambda x: '461-004-'+x[:3]+'-'+x[3:])
patients_461_004

<ipython-input-16-07d0c87f4202>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_461_004['patient_id'] = patients_461_004.patient_id.apply(lambda x: '461-004-'+x[:3]+'-'+x[3:])


,patient_id,ethnic,sex,race,dob
0,461-004-102-011,Not Hispanic or Latino,Male,White,1970-08-04
1,461-004-102-012,Not Hispanic or Latino,Female,White,1950-06-03
2,461-004-101-001,Not Hispanic or Latino,Male,White,1957-03-01
3,461-004-102-013,Not Hispanic or Latino,Female,White,1946-01-24
4,461-004-101-002,Not Hispanic or Latino,Female,White,1949-04-26
5,461-004-103-021,Not Hispanic or Latino,Male,White,1951-02-06
6,461-004-101-003,Not Hispanic or Latino,Male,White,1967-10-12
7,461-004-102-014,Not Hispanic or Latino,Male,White,1970-03-19
8,461-004-102-015,Not Hispanic or Latino,Male,White,1946-04-03
9,461-004-101-004,Not Hispanic or Latino,Female,White,1958-06-27


raceafr: African ancestry
raceeur: European or SW Asian (Turkey, Arabia, Iran) ancestry
racesa: South Asian ancestry
racepac: Pacific Islander ancestry
raceame: Native American ancestry
raceunk: Unknown ancestry
raceese: East and SE Asian ancestry

race2as: Asian
race2aia: American Indian or Alaskan Native
race2baa: Black or African American
race2nho: Native Hawaiian or other Pacific Islander
race2wh: White
race2unk: Unknown

In [17]:
patients_461_005['dob'] = patients_461_005.dob.apply(lambda x: pd.to_datetime(str(x)[:-2]+'0101',format='%Y%m%d'))
patients_461_005['raceoth'] = 'No Data'
patients_461_005.iloc[13,-1] = 'raceafr, raceame'
patients_461_005.iloc[101,-1] = 'raceeur, raceame'
patients_461_005.iloc[187,-1] = 'raceafr, raceeur'
patients_461_005.iloc[191,-1] = 'raceeur, raceame'
patients_461_005.iloc[292,-1] = 'raceeur, raceame'
patients_461_005.iloc[377,-1] = 'raceeur, raceame'
patients_461_005.iloc[385,-1] = 'raceeur, raceame'
patients_461_005.iloc[386,-1] = 'raceeur, raceame'
patients_461_005.iloc[13,7] = 0
patients_461_005.iloc[101,7] = 0
patients_461_005.iloc[187,4] = 0
patients_461_005.iloc[191,7] = 0
patients_461_005.iloc[292,7] = 0
patients_461_005.iloc[377,7] = 0
patients_461_005.iloc[385,7] = 0
patients_461_005.iloc[386,7] = 0
patients_461_005.iloc[63,8] = 1
patients_461_005.iloc[184,8] = 1
patients_461_005.iloc[338,8] = 1
patients_461_005.iloc[353,8] = 1
patients_461_005.iloc[338,-1] = 'raceese, race2as'
patients_461_005.iloc[353,-1] = 'raceese, race2as'
patients_461_005.iloc[338,9] = 0
patients_461_005.iloc[353,9] = 0
race_vars_005 = [col for col in patients_461_005.columns if 'race' in col]
race_vars_005.pop()
race_anc_005 = patients_461_005[['raceafr','raceeur','racepac','raceunk']].fillna(0)
race_anc_005.loc[race_anc_005.sum(axis=1)==0,'raceunk'] = 1
race_005 = patients_461_005[['raceese','race2as','race2aia','race2baa','race2nho','race2wh','race2unk']].fillna(0)
race_005.loc[race_005.sum(axis=1)==0,'race2unk'] = 1
race1_005 = pd.Series(race_anc_005.columns[np.where(race_anc_005!=0)[1]])
race2_005 = pd.Series(race_005.columns[np.where(race_005!=0)[1]])
patients_461_005 = pd.concat([patients_461_005,race1_005,race2_005],axis=1).drop(race_vars_005,axis=1)
patients_461_005 = patients_461_005.rename(columns={0:'race_ancstr',1:'race'})

In [18]:
patients_461_005
d_461_005.columns
d_461_015.columns

Index(['projectid', 'project', 'studyid', 'environmentName', 'subjectId',
       'StudySiteId', 'Subject', 'SDVTier', 'siteid', 'Site', 'SiteNumber',
       'SiteGroup', 'instanceId', 'InstanceName', 'InstanceRepeatNumber',
       'folderid', 'Folder', 'FolderName', 'FolderSeq', 'TargetDays',
       'DataPageId', 'DataPageName', 'PageRepeatNumber', 'RecordDate',
       'RecordId', 'RecordPosition', 'MinCreated', 'MaxUpdated', 'SaveTS',
       'CODER_HIERARCHY', 'AGE', 'AGE_RAW', 'AGEU', 'SEX', 'SEX_STD', 'ETHNIC',
       'ETHNIC_STD', 'RACE1', 'RACE1_RAW', 'RACE2', 'RACE2_RAW', 'RACE3',
       'RACE3_RAW', 'RACE4', 'RACE4_RAW', 'RACE5', 'RACE5_RAW', 'RACE6',
       'RACE6_RAW', 'RACEOTH'],
      dtype='object')

In [19]:
patients_461_006['raceoth'] = 'No Data'
patients_461_006.iloc[80,-1] = 'raceeur, raceamr'
patients_461_006.iloc[87,-1] = 'raceeur, raceamr'
patients_461_006.iloc[134,-1] = 'raceeur, raceamr'
patients_461_006.iloc[80,4] = 0
patients_461_006.iloc[87,4] = 0
patients_461_006.iloc[134,4] = 0
race_vars = [col for col in patients_461_006.columns if 'race' in col]
race_vars.pop()

<ipython-input-19-da33155f455a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_461_006['raceoth'] = 'No Data'
C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


'raceoth'

In [20]:
race_anc_006 = patients_461_006[['raceafr','raceeur','racesas','raceeas','racepac','raceamr','raceunk_x']]
race_006 = patients_461_006[['raceasian','raceamind','raceblack','racehawpac','racewhite','raceunk_y']]
race_006 = race_006.fillna(0)
race_006.loc[race_006.sum(axis=1)==0,'raceunk_y'] = 1
race1_006 = pd.Series(race_anc_006.columns[np.where(race_anc_006!=0)[1]])
race2_006 = pd.Series(race_006.columns[np.where(race_006!=0)[1]])
patients_461_006 = pd.concat([patients_461_006,race1_006,race2_006],axis=1).drop(race_vars,axis=1)
patients_461_006 = patients_461_006.rename(columns={0:'race_ancstr',1:'race'})

In [21]:
patients_461_007['dob'] = patients_461_007.dob.apply(lambda x: pd.to_datetime(str(x)[:-2]+'0101',format='%Y%m%d'))
patients_461_007['raceoth'] = 'No Data'
patients_461_007.iloc[121,-1] = 'raceeur, raceame'
patients_461_007.iloc[149,-1] = 'raceeur, raceame'
patients_461_007.iloc[121,4] = 0
patients_461_007.iloc[149,4] = 0
race_vars_007 = [col for col in patients_461_007.columns if 'race' in col]
race_vars_007.pop()

race_anc_007 = patients_461_007[['raceafr','raceeur','racepac','raceame','raceese','raceunk']]
race_anc_007 = race_anc_007.fillna(0)
race_anc_007.loc[race_anc_007.sum(axis=1)==0,'raceunk'] = 1

race_007 = patients_461_007[['race2as','race2aia','race2baa','race2nho','race2wh','race2unk']]
race_007 = race_007.fillna(0)
race_007.loc[race_007.sum(axis=1)==0,'race2unk'] = 1

race1_007 = pd.Series(race_anc_007.columns[np.where(race_anc_007!=0)[1]])
race2_007 = pd.Series(race_007.columns[np.where(race_007!=0)[1]])

patients_461_007 = pd.concat([patients_461_007,race1_007,race2_007],axis=1).drop(race_vars_007,axis=1)
patients_461_007 = patients_461_007.rename(columns={0:'race_ancstr',1:'race'})

In [22]:
race_vars = [col for col in patients_461_008.columns if 'race' in col]
race_anc = patients_461_008[['raceafr','raceeur','racesas','racepac','raceamr','raceunk']]
race = patients_461_008[['raceasian','raceamind','raceblack','racehawpac','racewhite','raceunk2']]

In [23]:
race1 = pd.Series(race_anc.columns[np.where(race_anc!=0)[1]])
race2 = pd.Series(race.columns[np.where(race!=0)[1]])
patients_461_008 = pd.concat([patients_461_008,race1,race2],axis=1).drop(race_vars,axis=1)
patients_461_008 = patients_461_008.rename(columns={0:'race_ancstr',1:'race'})

In [24]:
patients_461_012['random_no'] = patients_461_012.random_no.apply(lambda x: str(x))
patients_461_012['patient_id'] = patients_461_012.random_no.apply(lambda x: '461-012-1120-'+x[1:])
patients_461_012 = patients_461_012.drop(['random_no'],axis=1)
patients_461_012['dob'] = patients_461_012.dob.apply(lambda x: dt.date(2019,1,1)-dt.timedelta(days=x*365.24))

sex_461_012 = sex_461_012.rename(columns={'random_no':'patient_id','sbj_sex':'sex'})
sex_461_012['patient_id'] = sex_461_012.patient_id.apply(lambda x: '461-012-1120-'+str(x)[1:])
sex_461_012 = sex_461_012.drop_duplicates()

patients_461_012 = pd.merge(patients_461_012,sex_461_012,how='inner',on='patient_id')
patients_461_012

,patient_id,ethnic,race,dob,sex
0,461-012-1120-006,Not Hispanic or Latino,Asian - Korean,1976-01-02,Male
1,461-012-1120-011,Not Hispanic or Latino,Asian - Korean,1970-01-02,Male
2,461-012-1120-010,Not Hispanic,Asian - Japanese,1987-01-02,Male
3,461-012-1120-015,Not Hispanic or Latino,Asian,1985-01-01,Male
4,461-012-1120-009,Not Hispanic or Latino,Asian,1972-01-02,Male
5,461-012-1120-002,Not Hispanic or Latino,Asian - Korean,1960-01-02,Female
6,461-012-1120-005,Not Hispanic or Latino,Asian,1987-01-02,Male
7,461-012-1120-012,Not Hispanic or Latino,Asian- Chinese,1991-01-02,Female
8,461-012-1120-003,Not Hispanic or Latino,Asian,1991-01-02,Female
9,461-012-1120-008,Not Hispanic or Latino,Asian- Korean,1975-01-02,Female


In [25]:
patients_461_013['patient_id'] = patients_461_013.patient_id.apply(lambda x: '461-013-1130-'+x[-3:])
patients_461_013['age'] = patients_461_013.age.apply(lambda x: dt.date(2019,3,1)-dt.timedelta(days=x*365.24))
patients_461_013 = patients_461_013.rename(columns={'age':'dob'})

In [26]:
patients_461_014['dob'] = patients_461_014.dob.apply(lambda x: dt.date(2019,1,1)-dt.timedelta(days=x*365.24))
patients_461_014

,patient_id,ethnic,random_no,race,dob
0,461-014-34225,Not Hispanic or Latino,1008,White,1970-01-02
1,461-014-34290,Not Hispanic or Latino,1009,White,1970-01-02
2,461-014-63871,Not Hispanic or Latino,1024,Black or African American,1961-01-02
3,461-014-66754,Not Hispanic or Latino,1013,Asian,1985-01-01
4,461-014-67703,Not Hispanic or Latino,1018,White,1989-01-01
5,461-014-68978,Not Hispanic or Latino,1016,Black or African American,1992-01-02
6,461-014-69568,Not Hispanic or Latino,1001,Asian - Korean,1964-01-02
7,461-014-71717,Not Hispanic or Latino,1010,White,1961-01-02
8,461-014-74257,Not Hispanic or Latino,1002,White,1978-01-02
9,461-014-76451,Hispanic,1005,White,1990-01-02


race1: American Indian or Alaska Native
race2: Asian
race3: Black or African American
race4: Native Hawai'ian or other Pacific Islander
race5: White
race6: Other

In [27]:
patients_461_015['race'] = 'No Data'
patients_461_015 = patients_461_015.rename(columns={'race1':'raceamind','race2':'raceasian','race3':'raceblack','race4':'racehawpac','race5':'racewhite','race6':'raceunk_y'})
race_vars_015 = [col for col in patients_461_015.columns if 'race' in col]
race_015 = patients_461_015[['raceamind','raceasian','raceblack','racehawpac','racewhite','raceunk_y']]
race = pd.Series(race_015.columns[np.where(race_015!=0)[1]])
patients_461_015 = pd.concat([patients_461_015,race],axis=1).drop(race_vars_015,axis=1)
patients_461_015 = patients_461_015.rename(columns={0:'race'})
patients_461_015['patient_id'] = patients_461_015.patient_id.apply(lambda x: '461-015-115'+x[10]+x[-4:])
patients_461_015['age'] = patients_461_015.age.apply(lambda x: dt.date(2019,8,1)-dt.timedelta(days=x*365.24))
patients_461_015 = patients_461_015.rename(columns={'age':'dob'})

In [28]:
patients_461_015
# BREAK

,patient_id,ethnic,sex,dob,race
0,461-015-1152-001,Hispanic or Latino,Female,1950-08-02,racewhite
1,461-015-1152-013,Hispanic or Latino,Female,1954-08-02,racewhite
2,461-015-1151-001,Hispanic or Latino,Female,1981-08-01,racewhite
3,461-015-1151-004,Not Hispanic or Latino,Male,1961-08-02,racewhite
4,461-015-1151-006,Not Hispanic or Latino,Male,1957-08-02,raceblack
5,461-015-1152-016,Hispanic or Latino,Male,1968-08-01,racewhite
6,461-015-1151-008,Not Hispanic or Latino,Male,1951-08-02,racewhite
7,461-015-1151-007,Not Hispanic or Latino,Female,1971-08-02,raceblack
8,461-015-1151-010,Hispanic or Latino,Male,1954-08-02,racewhite
9,461-015-1151-012,Not Hispanic or Latino,Male,1966-08-02,racewhite


In [29]:
patients_461_018['dob'] = patients_461_018.dob.apply(handle_nan)
patients_461_018['dob'] = patients_461_018.dob.apply(lambda x: dt.date(2020,1,1)-dt.timedelta(days=x*365.24) if type(x)==float else x)
patients_461_018['patient_id'] = patients_461_018.patient_id.apply(lambda x: x[:11]+'-'+x[11:])
patients_461_018[patients_461_018.dob.isna()]
patients_461_018

sex_461_018['ID_PARTICIPATION'] = sex_461_018.ID_PARTICIPATION.apply(lambda x: '461-018-'+str(x)[:3]+'-'+str(x)[3:])
sex_461_018 = sex_461_018.rename(columns={'ID_PARTICIPATION':'patient_id','SBJ_SEX':'sex'}).drop_duplicates()

patients_461_018 = pd.merge(patients_461_018,sex_461_018,how='inner',on='patient_id')

In [30]:
patients_224_001['dob'] = patients_224_001.dob.apply(lambda x: x.date())
patients_461_004['dob'] = patients_461_004.dob.apply(lambda x: x.date())
patients_461_005['dob'] = patients_461_005.dob.apply(lambda x: x.date())
patients_461_006['dob'] = patients_461_006.dob.apply(lambda x: x.date())
patients_461_007['dob'] = patients_461_007.dob.apply(lambda x: x.date())
patients_461_008['dob'] = patients_461_008.dob.apply(lambda x: x.date())

<ipython-input-30-6e14bd3aa07b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_461_004['dob'] = patients_461_004.dob.apply(lambda x: x.date())


In [31]:
for i,row in patients_491_003_mad.iterrows():
    if patients_491_003_mad.loc[i,'race'] == 'OTHER':
        patients_491_003_mad.loc[i,'race'] = patients_491_003_mad.loc[i,'raceoth']
patients_491_003_mad = patients_491_003_mad.drop(['raceoth'],axis=1)

In [32]:
patients_491_003_mad['patient_id'] = patients_491_003_mad.patient_id.apply(lambda x: x[4:])
patients_491_003_sad['patient_id'] = patients_491_003_sad.patient_id.apply(lambda x: x[4:])

In [33]:
patients_491_003_mad['dob'] = patients_491_003_mad.dob.apply(lambda x: x.to_datetime64())
patients_491_003_sad['dob'] = patients_491_003_sad.dob.apply(lambda x: x.to_datetime64())

In [34]:
PATIENTS = pd.concat([patients_224_001,patients_461_004,patients_461_005,patients_461_006,patients_461_007,patients_461_008,patients_461_012,patients_461_013,patients_461_014,patients_461_015,patients_461_018,patients_491_003_mad,patients_491_003_sad],axis=0)
PATIENTS = PATIENTS.drop_duplicates(subset=['patient_id'])
print('This is the initial number of patients added into PATIENTS table from demographic data from','\n','224-001','\n','461-004','\n','461-005','\n','461-006','\n','461-007','\n','461-008','\n','461-012','\n','461-013','\n','461-014','\n','461-015','\n','461-018','\n','491-003 MAD/SAD:','\n',PATIENTS.shape[0])

This is the initial number of patients added into PATIENTS table from demographic data from 
 224-001 
 461-004 
 461-005 
 461-006 
 461-007 
 461-008 
 461-012 
 461-013 
 461-014 
 461-015 
 461-018 
 491-003 MAD/SAD: 
 868


In [35]:
PATIENTS2 = pd.merge(demog2_461_014,PATIENTS[['patient_id','dob']],how='left',on='patient_id')
PATIENTS2 = PATIENTS2.rename(columns={'gender':'sex','ethnicity':'ethnic'})
PATIENTS = PATIENTS[~PATIENTS.patient_id.str.contains('461-014-')]
PATIENTS = pd.concat([PATIENTS,PATIENTS2],axis=0)
print('This is number patients after adding demog data from 461-014: ',PATIENTS.shape[0])

This is number patients after adding demog data from 461-014:  890


In [36]:
PATIENTS['ethnic'] = PATIENTS.ethnic.apply(lambda x: x.lower() if type(x)==str else x)
PATIENTS = PATIENTS.replace({'ethnic':{'hispanic':'hispanic or latino','not hispanic':'not hispanic or latino'}})
PATIENTS = PATIENTS.rename(columns={'race_x':'race'})
PATIENTS['race'] = PATIENTS.race.apply(lambda x: x.lower() if type(x) == str else x)
PATIENTS = PATIENTS.replace({'race':{'asian- korean':'east or SE asian','asian - korean':'east or SE asian','race2wh':'white','racewhite':'white','race2unk':'unknown','unkown':'unknown','raceunk_y':'unknown','race2as':'asian','race2baa':'black or AA','black or african american':'black or AA','black':'black or AA','raceblack':'black or AA','other, specify':'other','mix asian and white':'other','raceese':'east or SE asian','asian- korean':'east or SE asian','asian - japanese':'east or SE asian','asian - korean':'east or SE asian','asian- chinese':'east or SE asian','east - japanese':'east or SE asian','race2aia':'american indian or alaska native','hispanic':'latino','south american':'latino','raceasian':'asian','african american':'black or AA','native hawaiian or other pacific islander':'native hawaiian or PI'},'sex':{'f':"F",'m':'M'}})

In [37]:
PATIENTS['raceoth'] = PATIENTS.raceoth.apply(handle_nan)
PATIENTS['race_ancstr'] = np.where(PATIENTS.raceoth != 'No Data',PATIENTS.raceoth,PATIENTS.race_ancstr)
PATIENTS = PATIENTS.drop(['raceoth'],axis=1)

In [38]:
# d_224_001['SubjectId']
# d_461_004['Subject']
# d1_461_005['Subject']
# d1_461_006['Subject']
# d_461_007['Subject']
# d_461_008['Subject']
# d_461_012['id_subject']
# d_461_013['USUBJID']
# d_461_014['id_subject']
# d_461_015['Subject']
# d_461_018['ID_SUBJECT']
# d_491_003_mad['USUBJID']
# d_491_003_sad['USUBJID']

In [39]:
PATIENTS = PATIENTS.replace({'sex':{'m':'M','f':'F','Male':'M','Female':'F','MALE':'M','FEMALE':'F'},'race_ancstr':{'raceeas':'east or SE asian','raceeur':'euro','raceunk':'unknown','raceafr':'afr','raceunk_x':'unknown','raceamr':'native','raceeur, raceame':'euro, native','raceeur, raceamr':'euro, native','raceese, race2as':'east or SE asian, asian','raceafr, raceeur':'afr, euro','raceafr, raceame':'afr, native'}})     
PATIENTS = PATIENTS.reset_index()
PATIENTS = PATIENTS.rename(columns={'index':'orig_idx'})

In [40]:
PATIENTS.shape
patients_before_add_sf = set(PATIENTS.patient_id.values)

In [41]:
# To add screen fail patients that were not included in patient demog data
PATIENTS = pd.merge(SCREEN_FAILS1,PATIENTS,how='outer',on='patient_id')
print("This is number patients after adding any patients from SCREEN_FAILS1 who do not already appear in PATIENTS: ",PATIENTS.shape[0])
PATIENTS = PATIENTS[['patient_id','ethnic','sex','race','dob','race_ancstr']]

This is number patients after adding any patients from SCREEN_FAILS1 who do not already appear in PATIENTS:  1180


In [42]:
patients_after_add_sf = set(PATIENTS.patient_id.values)
patients_added_from_sf = patients_after_add_sf - patients_before_add_sf 
# ONLY includes patients from 224-001 and 461-015
patients_added_from_sf 

{'224-001-100-001',
 '224-001-100-002',
 '224-001-100-003',
 '224-001-100-004',
 '224-001-100-005',
 '224-001-100-007',
 '224-001-100-008',
 '224-001-100-009',
 '224-001-100-010',
 '224-001-100-011',
 '224-001-100-013',
 '224-001-100-014',
 '224-001-100-015',
 '224-001-100-016',
 '224-001-100-018',
 '224-001-100-019',
 '224-001-100-020',
 '224-001-100-021',
 '224-001-100-022',
 '224-001-100-023',
 '224-001-100-024',
 '224-001-100-025',
 '224-001-100-026',
 '224-001-100-027',
 '224-001-100-028',
 '224-001-100-029',
 '224-001-100-030',
 '224-001-100-031',
 '224-001-100-032',
 '224-001-100-033',
 '224-001-100-034',
 '224-001-100-035',
 '224-001-100-036',
 '224-001-100-037',
 '224-001-100-038',
 '224-001-100-039',
 '224-001-100-040',
 '224-001-100-041',
 '224-001-100-043',
 '224-001-100-044',
 '224-001-100-048',
 '224-001-100-049',
 '224-001-100-050',
 '224-001-100-051',
 '224-001-100-053',
 '224-001-100-054',
 '224-001-100-055',
 '224-001-100-056',
 '224-001-100-057',
 '224-001-100-058',


In [43]:
patients_from_visits = PATIENT_VISITS.patient_id.drop_duplicates()
patients_from_visits = pd.DataFrame({'patient_id':patients_from_visits})

In [44]:
print(len(set(PATIENTS.patient_id.values) & set(PATIENT_VISITS.patient_id.values)))
print('Number patients from PATIENTS: ',PATIENTS.shape[0])
print('Unique number patients from PATIENT_VISITS: ',len(PATIENT_VISITS.patient_id.value_counts()))
print('Number patients from patients_from_visits: ',patients_from_visits.shape[0])

1179
Number patients from PATIENTS:  1180
Unique number patients from PATIENT_VISITS:  1308
Number patients from patients_from_visits:  1308


In [45]:
print('Number patients before adding patients_from_visits: ',PATIENTS.shape[0])
PATIENTS = pd.merge(patients_from_visits,PATIENTS,how='outer',on='patient_id')
print('Number patients after adding patients_from_visits: ',PATIENTS.shape[0])
print('This difference indicates patients who were included in visits logs, but who were not added into PATIENTS from demographic data OR SCREEN_FAILS data.')

Number patients before adding patients_from_visits:  1180
Number patients after adding patients_from_visits:  1309
This difference indicates patients who were included in visits logs, but who were not added into PATIENTS from demographic data OR SCREEN_FAILS data.


In [46]:
def calc_age(df):
    for i,row in df.iterrows():
        if type(row.dob) != float:
            df.loc[i,'age_in_days'] = np.datetime64(dt.datetime.now())-row.dob
    return df

def calc_age_group(df):
    df['age_group'] = df.age_in_days / 365.24
    for i,row in df.iterrows():
        if row.age_group >= 80.0:
            df.loc[i,'age_group'] = '80+'
        if row.age_group < 80.0 and row.age_group >= 70.0:
            df.loc[i,'age_group'] = '70-79'
        if row.age_group < 70.0 and row.age_group >= 60.0:
            df.loc[i,'age_group'] = '60-69'
        if row.age_group < 60.0 and row.age_group >= 50.0:
            df.loc[i,'age_group'] = '50-59'
        if row.age_group < 50.0 and row.age_group >= 40.0:
            df.loc[i,'age_group'] = '40-49'
        if row.age_group < 40.0 and row.age_group >= 30.0:
            df.loc[i,'age_group'] = '30-39'
        if row.age_group < 30.0:
            df.loc[i,'age_group'] = 'Under 30'
    return df
        
patients_age = PATIENTS.dropna(subset=['dob'],axis=0)
patients_age['dob'] = pd.to_datetime(patients_age.dob)
patients_age = calc_age(patients_age)
patients_age['age_in_days'] = patients_age.age_in_days.apply(lambda x: pd.to_timedelta(x,unit='d')/np.timedelta64(1,'D'))
patients = calc_age_group(patients_age)
patients_age.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/age_filtered_patients.csv')
PATIENTS['dob'] = pd.to_datetime(PATIENTS.dob)
PATIENTS = calc_age(PATIENTS)
PATIENTS['age_in_days'] = PATIENTS.age_in_days.apply(lambda x: pd.to_timedelta(x,unit='d')/np.timedelta64(1,'D'))
PATIENTS = calc_age_group(PATIENTS)

<ipython-input-46-92a9a4ca6473>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_age['dob'] = pd.to_datetime(patients_age.dob)
C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [47]:
PATIENTS

,patient_id,ethnic,sex,race,dob,race_ancstr,age_in_days,age_group
0,224-001-100-001,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,224-001-100-002,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,224-001-100-003,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,224-001-100-004,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,224-001-100-005,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1304,491-003-109-112,not hispanic or latino,F,white,1947-03-01,,26863.577336,70-79
1305,491-003-109-113,not hispanic or latino,M,white,1961-04-11,,21708.577336,50-59
1306,491-003-111-101,not hispanic or latino,F,white,1948-08-13,,26332.577336,70-79
1307,491-003-401-101,not hispanic or latino,M,white,1963-04-28,,20961.577336,50-59


In [48]:
PATIENTS.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/all_patients.csv',date_format='%Y%m%d')

In [49]:
print('The number of patients that are common in patients and patient visits: ',len(set(PATIENTS.patient_id.values) & set(PATIENT_VISITS.patient_id.values)))
print('The patients table has a count of: ',PATIENTS.shape[0])
print('The patient visits table has a unique count of: ',len(PATIENT_VISITS.patient_id.unique()))
ids_from_PATIENTS = set(PATIENTS.patient_id.values)
ids_from_PATIENT_VISITS = set(PATIENT_VISITS.patient_id.values)
ids_missing_from_PVs = ids_from_PATIENTS - ids_from_PATIENT_VISITS
print('These are the patients who were added into PATIENTS but do not appear in PATIENT_VISITS: ',ids_missing_from_PVs)

The number of patients that are common in patients and patient visits:  1308
The patients table has a count of:  1309
The patient visits table has a unique count of:  1308
These are the patients who were added into PATIENTS but do not appear in PATIENT_VISITS:  {'461-005-902-101'}


In [50]:
test = PATIENTS[(PATIENTS.race.isna()) | (PATIENTS.race == 'unknown')]
test2 = PATIENTS[(PATIENTS.race.notna()) & (PATIENTS.race != 'unknown')]
test3 = PATIENTS[PATIENTS.race.notna()]
oo1 = 0
oo4 = 0
oo5 = 0
oo6 = 0
oo8 = 0
o12 = 0
o13 = 0
o14 = 0
o15 = 0
o18 = 0
oo3 = 0
for i,row in test3.iterrows():
    if row.patient_id[:7] == '224-001':
        oo1 += 1
    if row.patient_id[:7] == '461-004':
        oo4 += 1
    if row.patient_id[:7] == '461-005':
        oo5 += 1
    if row.patient_id[:7] == '461-006':
        oo6 += 1
    if row.patient_id[:7] == '461-008':
        oo8 += 1
    if row.patient_id[:7] == '461-012':
        o12 += 1
    if row.patient_id[:7] == '461-013':
        o13 += 1
    if row.patient_id[:7] == '461-014':
        o14 += 1
    if row.patient_id[:7] == '461-015':
        o15 += 1
    if row.patient_id[:7] == '461-018':
        o18 += 1        
    if row.patient_id[:7] == '491-003':
        oo3 += 1
oo5

437

DATA QUALITY CHECK

In [51]:
PATIENTS[(PATIENTS.patient_id.str.contains('461-015'))&(PATIENTS.age_group.notna())].shape
# PATIENTS.patient_id.value_counts()
# PATIENTS[PATIENTS.patient_id=='461-014-91819']
# PATIENTS

(27, 8)

In [52]:
all_patients = set(PATIENTS.patient_id.values)
all_patient_visits = set(PATIENT_VISITS.patient_id.values)
not_in_pvs = all_patients - all_patient_visits
not_in_pvs
not_in_patients = all_patient_visits - all_patients

# patients_007 = set(PATIENTS[PATIENTS.patient_id.str.contains('461-007')].patient_id.values)
# pvs_007 = set(PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_007'].patient_id.values)
# pvs_007 - patients_007
# PATIENTS[PATIENTS.patient_id=='461-005-101-502']
not_in_pvs

{'461-005-902-101'}

In [53]:
PATIENTS[PATIENTS.patient_id.str.contains('224-001')]
demog_224_001
# d_461_012
# demog_461_012
# PATIENTS

,patient_id,dob,age,sex,dmcbp,dmcbpcd,f_dmbcm,f_dmbcmcd,m_dmbcm,m_dmbcmcd,ethnic,race,raceoth,dmess,dmesscd,dmrdyn,dmrdyncd
0,100-006,1993,25,Female,Yes,1.0,Barrier plus hormonal contraception or intraut...,1.0,NaN,NaN,Not Hispanic or Latino,ASIAN,NaN,Yes,1,Yes,1
1,100-012,1996,23,Male,NaN,NaN,NaN,NaN,Abstain from sexual activity,3.0,Not Hispanic or Latino,WHITE,NaN,Yes,1,Yes,1
2,100-017,1992,26,Female,Yes,1.0,Barrier plus hormonal contraception or intraut...,1.0,NaN,NaN,Hispanic or Latino,OTHER,South american,Yes,1,Yes,1
3,100-042,1990,28,Female,Yes,1.0,Barrier plus hormonal contraception or intraut...,1.0,NaN,NaN,Not Hispanic or Latino,WHITE,NaN,Yes,1,Yes,1
4,100-052,1997,22,Female,Yes,1.0,Barrier plus hormonal contraception or intraut...,1.0,NaN,NaN,Not Hispanic or Latino,WHITE,NaN,Yes,1,Yes,1
5,100-046,1991,27,Male,NaN,NaN,NaN,NaN,"Barrier plus hormonal contraception (eg, male ...",2.0,Not Hispanic or Latino,WHITE,NaN,Yes,1,Yes,1
6,100-047,1984,34,Male,NaN,NaN,NaN,NaN,"Barrier plus hormonal contraception (eg, male ...",2.0,Not Hispanic or Latino,WHITE,NaN,Yes,1,Yes,1
7,100-045,1995,24,Female,Yes,1.0,Barrier plus hormonal contraception or intraut...,1.0,NaN,NaN,Not Hispanic or Latino,BLACK OR AFRICAN AMERICAN,NaN,Yes,1,Yes,1
8,100-067,1988,31,Male,NaN,NaN,NaN,NaN,Abstain from sexual activity,3.0,Not Hispanic or Latino,ASIAN,NaN,Yes,1,Yes,1
9,100-092,1988,31,Female,Yes,1.0,Barrier plus hormonal contraception or intraut...,1.0,NaN,NaN,Not Hispanic or Latino,WHITE,NaN,Yes,1,Yes,1


In [54]:
PATIENTS[PATIENTS.patient_id.str.contains('461-005')].patient_id.value_counts()
# have_dm_data = PATIENTS[(PATIENTS.patient_id.str.contains('461-005'))&(PATIENTS.ethnic.notna())]
# demog_461_005[demog_461_005.patient_id=='461-005-902-101']
PATIENTS[PATIENTS.patient_id=='461-005-902-501']
# SCREEN_FAILS1[SCREEN_FAILS1.patient_id=='461-005-902-101']
# PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-902-101']
# d_461_005[d_461_005.Subject=='461-005-902-101']
# ids_not_in_demog = set(have_dm_data.patient_id.values) - set(demog_461_005.patient_id.values)
# PATIENTS[PATIENTS.patient_id=='461-005-902-101'] -- this patient does not appear in visits log
# d1_461_005[d1_461_005.Subject=='461-005-902-101']
# d2_461_005[d2_461_005.Subject=='461-005-902-101']
# patients_461_005[patients_461_005.patient_id=='461-005-902-101']
# patients_from_visits[patients_from_visits.patient_id=='461-005-902-101']
d_461_007[d_461_007.Subject=='461-005-297-501']
# PATIENTS[PATIENTS.patient_id=='461-005-297-101']
# demog_461_005[demog_461_005.patient_id.str.contains('461-005-297')]
patients_461_014
# print(demog_461_014.shape)
# print(demog2_461_014.shape)
# demog_461_014.columns
PATIENTS[(PATIENTS.patient_id.str.contains('461-014'))&(PATIENTS.dob.notna())].shape
PATIENTS[PATIENTS.patient_id=='461-018-102-866']

,patient_id,ethnic,sex,race,dob,race_ancstr,age_in_days,age_group
1220,461-018-102-866,not hispanic or latino,F,black or AA,1987-01-02,,12311.577335,30-39


In [55]:
# In 461-007, these are the patient IDs that do not match from 461-005
# d_461_007.loc[219,'Subject'] = '461-005-902-501'
# d_461_007.loc[224,'Subject'] = '461-005-297-502'
# d_461_007.loc[225,'Subject'] = '461-005-297-501'